# Working with Databricks outside of the UI

## Lets query the ingested data

In [ ]:
from pyspark.sql import DataFrame, SparkSession
import pandas as pd

In [ ]:
# Spark here is handled by the databricks-connect library
# 
# Once installed it's configured by calling "databricks-connect configure" where we provide details on how
# to connect to our cluster. The version of databricks-connect installed must match the Databricks Runtime
# version on the cluster

spark: SparkSession = SparkSession.builder.getOrCreate()

asset_id: int = int(spark.sql("SELECT MAX(AssetNumber) FROM silver.events").head()[0])

df: DataFrame = spark.sql(f"SELECT * FROM silver.events WHERE AssetNumber = {asset_id}")

df.show()

## Now lets query the data, aggregate it, and pull it back locally as a Pandas DataFrame so that we can plot it

In [ ]:
# Now lets read the data for the asset and bucket the data into hours with the average
# value for the hour

grouped_df: DataFrame = spark.sql(("SELECT "
                                  "    AssetNumber "
                                  "    , DATE_TRUNC('HOUR', EventDate) AS EventDateHour "
                                  "    , AVG(Value) AS AverageValue "
                                  "FROM "
                                  "    silver.events "
                                  "WHERE "
                                  f"    AssetNumber = {asset_id} "
                                  "GROUP BY "
                                  "    AssetNumber "
                                  "    , EventDateHour "
                                  "ORDER BY "
                                  "    EventDateHour"))

grouped_pd_df: pd.DataFrame = grouped_df.toPandas()

In [ ]:
pd.options.plotting.backend = "matplotlib"
fig = grouped_pd_df.plot(x='EventDateHour', y='AverageValue', figsize=(20, 7))